# Red Inicial


## Imports


In [83]:
from collections import deque
from enum import Enum
from functools import cache
from random import random
from time import perf_counter, time
from typing import Generator, Optional

import gymnasium as gym
import numpy as np
from gymnasium.spaces import Box, Dict, Discrete
from gymnasium.utils import seeding
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

## Clases y Funciones


In [84]:
class Packet_Generator():
    def __init__(self, min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=3, max_size=20,
                 min_rate=0, max_rate=4):

        self.packet = Dict({
            "IP":  Box(low=min_ip, high=max_ip, shape=(), dtype=int),
            "PORT":  Box(low=min_port, high=max_port, shape=(), dtype=int),
            "PROTOCOL":  Box(low=min_protocol, high=max_protocol, shape=(), dtype=int),
            "SIZE":  Box(low=min_size, high=max_size, shape=(), dtype=int)
        })
        self.min_rate: int = min_rate
        self.max_rate: int = max_rate

    def generate_packet(self):
        return self.packet.sample()

    def generate_packets(self):
        num_packets: int = np.random.randint(self.min_rate, self.max_rate)
        return [self.generate_packet() for _ in range(num_packets)]


class DOS_Packet_Generator(Packet_Generator):
    def __init__(self,
                 min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=5, max_size=15,
                 min_rate=2, max_rate=10):

        ip: int = np.random.randint(min_ip, max_ip)
        super().__init__(ip, ip,
                         min_port, max_port,
                         min_protocol, max_protocol,
                         min_size, max_size,
                         min_rate, max_rate)


class DDOS_Packet_Generator(Packet_Generator):
    def __init__(self,
                 min_ip=0, max_ip=2000,
                 min_port=0, max_port=4000,
                 min_protocol=0, max_protocol=100,
                 min_size=5, max_size=15,
                 min_rate=2, max_rate=10):

        super().__init__(min_ip, max_ip,
                         min_port, max_port,
                         min_protocol, max_protocol,
                         min_size, max_size,
                         min_rate, max_rate)


class PacketAttack(Enum):
    @staticmethod
    def new_set(description, weight, class_ref):
        return {
            "Description": description,
            "weight": weight,
            "class": class_ref
        }

    @staticmethod
    def not_implemented():
        raise NotImplementedError(f"Class not implemented")

    @classmethod
    @cache
    def weights(cls):
        attack_weights = []
        for attack in PacketAttack:
            attack_weights.append(attack.value["weight"])
        return np.array(attack_weights)

    # ----ENUM VALUES----
    DOS = new_set("Denial of Service", 1.0, DOS_Packet_Generator)
    DDOS = new_set("Distributed Denial of Service", 2.0, DDOS_Packet_Generator)


dos_gen = DOS_Packet_Generator()

print(dos_gen.generate_packets())

[{'IP': 1136, 'PORT': 705, 'PROTOCOL': 36, 'SIZE': 8}, {'IP': 1136, 'PORT': 3805, 'PROTOCOL': 87, 'SIZE': 11}, {'IP': 1136, 'PORT': 1106, 'PROTOCOL': 11, 'SIZE': 12}, {'IP': 1136, 'PORT': 3170, 'PROTOCOL': 4, 'SIZE': 11}, {'IP': 1136, 'PORT': 3558, 'PROTOCOL': 16, 'SIZE': 9}, {'IP': 1136, 'PORT': 2086, 'PROTOCOL': 94, 'SIZE': 6}, {'IP': 1136, 'PORT': 975, 'PROTOCOL': 15, 'SIZE': 5}, {'IP': 1136, 'PORT': 3827, 'PROTOCOL': 91, 'SIZE': 10}, {'IP': 1136, 'PORT': 2600, 'PROTOCOL': 8, 'SIZE': 5}]


In [85]:
from time import perf_counter
tiempo_global: float = 0.0
iteraciones = 0


def medir_tiempo(activado=False):
    def fun(funcion):
        def wrapper(*args, **kwargs):
            fun_tiempo = perf_counter
            inicio: float = fun_tiempo()
            resultado = funcion(*args, **kwargs)
            if not activado:
                return resultado
            tiempo_total: float = fun_tiempo() - inicio
            global tiempo_global, iteraciones
            medida = 1e6
            t: float = tiempo_total*medida
            tiempo_global += t
            iteraciones += 1
            print(f"Tiempo de ejecución de {
                funcion.__name__}: {t:.2f} micro-segundos")
            return resultado
        return wrapper
    return fun


In [86]:
class Acciones(Enum):
    PERMITIR = 0
    DENEGAR = 2

    @classmethod
    def int_to_action(cls, action: int) -> "Acciones":
        return cls._get_actions_list()[action]

    @classmethod
    @cache
    def _get_actions_list(cls):
        return list(Acciones)


class RouterEnv(gym.Env):
    total_time: float = 100.0 # En microsegundos

    def __init__(self, max_len=20, seed: Optional[int] = None):

        super(RouterEnv, self).__init__()

        self.max_len: int = max_len
        self.rate = 5  # bytes por segundo de procesamiento
        self.attack_probability = 0.0

        self._set_initial_values(seed)

        self.observation_space = Box(low=0, high=self.max_len, shape=(
            len(self.calculate_queue_stats()),), dtype=np.uint16)  # Paquetes en la cola

        self.action_space = Discrete(len(Acciones))

    def _set_initial_values(self, seed):
        self.queue = deque(maxlen=self.max_len)
        self.mb_restantes: float = 0.0
        self.step_durations: list[float] = []
        self._np_random, self._np_random_seed = seeding.np_random(seed)
        self.state: Acciones = Acciones.PERMITIR
        self.uds_tiempo_pasado: float = 0.0

    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        self._set_initial_values(seed)
        
        observation = self._get_obs()
        info = self._get_info()
        return observation, info

    def _get_obs(self):
        stats = self.calculate_queue_stats()
        return stats

    def _get_info(self):
        return {
            "Queue": np.array(self.queue),
            "AttackProb": self.attack_probability
        }

    def calculate_queue_stats(self):
        tam_total = 0
        for paquete in self.queue:
            tam_total += paquete["SIZE"]
        num_packets: int = len(self.queue)
        return np.array([num_packets, tam_total], dtype=np.uint16)

    def packet_input(self) -> int:
        if self.state == Acciones.PERMITIR:
            prob: float = self.np_random.random()
            if prob > self.attack_probability:
                # Generación de paquetes normales
                paquetes = Packet_Generator().generate_packets()
            else:
                # Generación de paquetes maliciosos
                paquetes = DOS_Packet_Generator().generate_packets()
            if len(self.queue) + len(paquetes) > self.max_len:
                espacio_libre= self.max_len - len(self.queue)
                self.queue.extend(paquetes[:espacio_libre])
                return len(paquetes) - (espacio_libre)
            self.queue.extend(paquetes)
            return 0

    @medir_tiempo(False)
    def step(self, action: int):
        # Prob
        # Hacer procesado por tamaño de paquete
        # Cada paso debería ser 10 microsegundos

        #Suponiendo que el paso es de 10 microsegundos:
        #start_time: float = time()

        action = Acciones.int_to_action(action)
        descartados: int=self.packet_input()
        self.procesar_por_tamaño()
        reward: float= self.get_reward(descartados,action)
        observation = self._get_obs()
        truncated = False # True si se desvía del comportamiento normal para abortar, necesitaría un reset
        info = self._get_info()

        self.uds_tiempo_pasado += 1
        finished: bool = self._is_finished_execution()

        #self.step_durations.append(time() - start_time)
        return observation, reward, finished, truncated, info
    
    def get_reward(self,descartados,action)->float:
        reward = 0.0
        espacio_libre: int = self.max_len - len(self.queue)
        reward += espacio_libre
        penalizacion_descartados=2
        reward-=descartados*penalizacion_descartados
        reward -= action.value
        return reward
    
    @medir_tiempo(0)
    def procesar_por_tamaño(self) :
                
        if len(self.queue) == 0:
            return 
        
        print(self.queue)
        tam_procesado = 0.0
        paquete = self.queue[0]
        # Calcula los mb que faltan por procesar
        self.mb_restantes = paquete["SIZE"]

        while tam_procesado < self.rate and len(self.queue) > 0:
            print(f"{self.mb_restantes} MB restantes")
            if self.mb_restantes == 0:
                p2=self.queue.popleft() # Quita el paquete que se ha procesado
                if len(self.queue) == 0:
                    break
                # Nuevo paquete
                paquete = self.queue[0]
                assert p2 != paquete
                # Calcula los mb que faltan por procesar
                self.mb_restantes = paquete["SIZE"]
            else:
                # Procesar
                procesado_local: float= min(self.mb_restantes, # Procesar lo que queda del paquete
                                            self.rate-tam_procesado) # Procesar lo que queda del paso
                self.mb_restantes -= procesado_local

        print(f"Procesado {tam_procesado} bytes")

        

    def _is_finished_execution(self) -> bool:
        # Terminar solo después de 10 pasos
        return self.uds_tiempo_pasado >= self.total_time
        return len(self.step_durations) >= 10
    
    def close(self):
        # Cerrar el entorno, liberar recursos, cerrar conexiones, etc
        return super().close()
    
    def render(self, mode='human'):
        # Renderizar el entorno
        return super().render(mode=mode)

In [87]:
env = RouterEnv(seed=1)
check_env(env)

model = PPO("MlpPolicy", env, verbose=True)

model.learn(total_timesteps=10)
model.save("Example")

deque([{'IP': 55, 'PORT': 691, 'PROTOCOL': 75, 'SIZE': 15}, {'IP': 1330, 'PORT': 3072, 'PROTOCOL': 10, 'SIZE': 18}, {'IP': 1642, 'PORT': 2209, 'PROTOCOL': 76, 'SIZE': 20}], maxlen=20)
15 MB restantes
10.0 MB restantes
5.0 MB restantes
0.0 MB restantes
18 MB restantes
13.0 MB restantes
8.0 MB restantes
3.0 MB restantes
0.0 MB restantes
20 MB restantes
15.0 MB restantes
10.0 MB restantes
5.0 MB restantes
0.0 MB restantes
Procesado 0.0 bytes
Tiempo de ejecución de procesar_por_tamaño: 324.50 micro-segundos
deque([{'IP': 461, 'PORT': 943, 'PROTOCOL': 19, 'SIZE': 9}], maxlen=20)
9 MB restantes
4.0 MB restantes
0.0 MB restantes
Procesado 0.0 bytes
Tiempo de ejecución de procesar_por_tamaño: 80.70 micro-segundos
Tiempo de ejecución de procesar_por_tamaño: 1.00 micro-segundos
deque([{'IP': 783, 'PORT': 87, 'PROTOCOL': 59, 'SIZE': 12}, {'IP': 1436, 'PORT': 920, 'PROTOCOL': 41, 'SIZE': 6}, {'IP': 1015, 'PORT': 2791, 'PROTOCOL': 59, 'SIZE': 15}], maxlen=20)
12 MB restantes
7.0 MB restantes
2.0 MB

In [88]:
print(f"Tiempo medio: {tiempo_global/iteraciones if abs(iteraciones)>1e-5 else 0:.2f} micro-segundos de {iteraciones} iteraciones")

Tiempo medio: 87.95 micro-segundos de 2059 iteraciones


In [89]:
#Pruebas 
cola=deque(maxlen=10)
cola.append(1)
cola.extend([2]*20)
print(cola)

deque([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], maxlen=10)
